<a href="https://colab.research.google.com/github/noahmott/yelpscraper/blob/main/YelpScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from bs4 import BeautifulSoup
import requests
import urllib
from datetime import datetime
import pandas as pd
import time
import utils.scraperagent

In [21]:

class YelpScraper(ScraperAgent):

  headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
  
  def __init__(self):
    self.scrapertool=ScraperAgent(self.headers)

  def scrape_data(self, url, set_range=8, current_date_css_tag='css-chan6m', current_review_css_tag='raw__09f24__T4Ezm'):
    urllist=self.__collect_urls(url, set_range)
    #print(urllist)
    dates2=[]
    reviews=[]
    for url in urllist:
      time.sleep(3)
      myurl=urllib.request.urlopen(url)
      soup=BeautifulSoup(myurl, 'html.parser')
      dates=soup.find_all('span', class_=current_date_css_tag)
      review=soup.find_all('span', class_=current_review_css_tag)
      for x in dates:
        try:
          datetimeobject=datetime.strptime(str(x.text), '%m/%d/%Y').date()
          dates2.append(datetimeobject)
        except Exception as e:
          pass
      for i in review[5::]:
        reviews.append(i.text)
    print('Dates: '+str(len(dates2)))
    print('Reviews: '+str(len(reviews)))
    if len(dates2)!=len(reviews):
      print('Check your data. You may have to run this again. Reviews and Dates are uneven')
    else:
      print('Data is even. All good!')
    constructeddf=pd.DataFrame()
    constructeddf['Date']=dates2
    constructeddf['Review']=pd.Series(reviews)
    return constructeddf

      
    
  def __collect_urls(self, url, set_range):   
    print('testing connection....')
    #print(type(self.scrapertool.testconnection(url)))
    if self.scrapertool.testconnection(url).status_code==200:
      print('Connection Good!')
      x=0
      urls=[url]
      for page in range(set_range):
        x+=10
        urls.append(url+'?start='+str(x))
      return urls



In [22]:
scraper=YelpScraper()

In [15]:
fulldf=scraper.scrape_data(url='https://www.yelp.com/biz/social-house-orlando-orlando')

testing connection....
Connection Good!
Dates: 95
Reviews: 98
Check your data. You may have to run this again. Reviews and Dates are uneven


In [171]:
fulldf

,Date,Review
0,2022-11-03,"Soooooo, our second visit to this place within..."
1,2022-09-27,Very racist. Don't go if you aren't white.Empl...
2,2022-02-04,My friend and I tried out Social House for the...
3,2022-08-06,Can I express how amazing the service here is!...
4,2022-09-24,"Lemme make this quick, good place to DRINK! No..."
...,...,...
90,2021-04-20,"If I could rate this place a 0,.......I would...."
91,2020-04-16,If I could Rui's place 0 I would. It's horribl...
92,2016-03-11,As a long time customer of the old establishme...
93,2016-03-16,Such a let down.I have been here twice for hap...
